In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [2]:
def transition_probability(curr_state, next_state, action):
    action_value = [-1, -10, 1, 10] # curr_state - next_state (based on state space)
    col = int(curr_state/10)
    row = int(curr_state%10)
    unallowed = [] # unallowed directions for a given state
    if row==0:
        unallowed.append(0) # top
    elif row==9:
        unallowed.append(2) # bottom
    if col==0:
        unallowed.append(1) # left
    elif col==9:
        unallowed.append(3) # right
    
    value = action_value[action]
    neighborhood = []
    for index in np.arange(0,4): # building neighborhood of the current state
        if index not in unallowed:
            neighborhood.append(curr_state + action_value[index])
    if unallowed: # not empty => edges or corners
        if len(unallowed)==2: # corners
            if action in unallowed: # action to move off the grid
                if next_state==curr_state: # => same_state
                    return 1-w+w/4+w/4
                elif next_state in neighborhood: # neighboring valid states
                    return w/4
            else: # action to stay in the grid
                if next_state==curr_state: # => same_state
                    return w/4+w/4
                elif next_state==curr_state+value and next_state in neighborhood: # intended state
                    return 1-w+w/4
                elif next_state in neighborhood: # neighboring valid states
                    return w/4      
        else: # edges
            if action in unallowed: # action to move off the grid
                if next_state==curr_state: # => same_state
                    return 1-w+w/4
                elif next_state in neighborhood: # neighboring valid states
                    return w/4
            else: # action to stay in the grid
                if next_state==curr_state: # => same_state
                    return w/4
                elif next_state==curr_state+value and next_state in neighborhood: # intended state
                    return 1-w+w/4
                elif next_state in neighborhood: # neighboring valid states
                    return w/4
            
    else: # non boundary
        if next_state==curr_state + value and next_state in neighborhood:
            return 1-w+w/4
        elif next_state in neighborhood:
            return w/4
    return 0

def construct_neighborhood(curr_state):
    col = int(curr_state/10)
    row = int(curr_state%10)
    action_value = [-1, -10, 1, 10] # curr_state - next_state (based on state space)
    unallowed = [] # unallowed directions for a given state
    if row==0:
        unallowed.append(0) # top
    elif row==9:
        unallowed.append(2) # bottom
    if col==0:
        unallowed.append(1) # left
    elif col==9:
        unallowed.append(3) # right
    
    neighborhood = []
    for index in np.arange(0,4): # building neighborhood of the current state
        if index not in unallowed:
            neighborhood.append(curr_state + action_value[index])
    return neighborhood

def value_iteration_action(reward, state_space, action_set, discount_factor):
    epsilon = 0.01
    V = np.zeros(shape=[10,10])
    pi = np.zeros(shape=[10,10])
    delta = float("inf")
    state_space=np.transpose(state_space)
    flatten_state = []
    for element in state_space:
        for el in element:
            flatten_state.append(el)
            
    maximum = -1*float("inf")
    while delta > epsilon:
        delta = 0
        temp=np.zeros([10,10])
        for curr_state in flatten_state:
            curr_col = int(curr_state/10)
            curr_row = int(curr_state%10)
            v = V[curr_row][curr_col]
            neighborhood = construct_neighborhood(curr_state)
            states_to_visit = neighborhood + [curr_state]
            maximum = -1*float("inf")
            for action in action_set:
                total = 0
                for next_state in states_to_visit:
                    p = transition_probability(curr_state, next_state, action)
                    next_col = int(next_state/10)
                    next_row = int(next_state%10)
                    total += p * (reward[next_row, next_col] + discount_factor * V[next_row][next_col])
                if total>maximum:
                    maximum = total
            temp[curr_row][curr_col] = maximum
            delta = max(delta, np.abs(v - temp[curr_row][curr_col]))
        V=temp
    maximum = -1*float("inf")
    for curr_state in flatten_state:
        curr_col = int(curr_state/10)
        curr_row = int(curr_state%10)
        neighborhood = construct_neighborhood(curr_state)
        states_to_visit = neighborhood + [curr_state]
        maximum = -1*float("inf")
        for action in [0,1,3,2]: # or action set
            total = 0
            for next_state in states_to_visit:
                p = transition_probability(curr_state, next_state, action)
                next_col = int(next_state/10)
                next_row = int(next_state%10)
                total += p * ((reward[next_row, next_col]) + discount_factor * V[next_row][next_col])
            if total> maximum:
                maximum = total
                max_index = action
        pi[curr_row][curr_col] = max_index
    return pi

In [3]:
def value_iteration_value(reward, state_space, action_set, discount_factor):
    epsilon = 0.01
    V = np.zeros(shape=[10,10])
    delta = float("inf")
    flatten_state = []
    for element in state_space:
        for el in element:
            flatten_state.append(el)
            
    maximum = -1*float("inf")
    while delta > epsilon:
        delta = 0 
        temp=np.zeros([10,10])
        for curr_state in flatten_state:
            curr_col = int(curr_state/10)
            curr_row = int(curr_state%10)
            v = V[curr_row][curr_col]
            neighborhood = construct_neighborhood(curr_state)
            states_to_visit = neighborhood + [curr_state]
            maximum = -1*float("inf")
            for action in action_set:
                total = 0
                for next_state in states_to_visit:
                    p = transition_probability(curr_state, next_state, action)
                    next_col = int(next_state/10)
                    next_row = int(next_state%10)
                    total += p * (reward[next_row, next_col] + discount_factor * V[next_row][next_col])
#                     if curr_row==9 and curr_col==9:
#                         print next_state, next_row, next_col
#                         print action, next_state, p, reward[next_row, next_col], V[next_row][next_col] 
                if total>maximum:
                    maximum = total
            temp[curr_row][curr_col] = maximum
            delta = max(delta, np.abs(v - temp[curr_row][curr_col]))
        V=temp.copy()
            
#             if curr_row==9 and curr_col==9:
#                 print ("currrow, currcol, max: ", curr_row, curr_col, maximum)
#             print ("currrow, currcol, max: ", curr_row, curr_col, maximum)
            
    return V
                    

In [4]:
def value_iteration_action1(reward, state_space, action_set, discount_factor):
    epsilon = 0.01
    V = np.zeros(shape=[10,10])
    pi = np.zeros(shape=[10,10])
    delta = float("inf")
    flatten_state = []
    for element in state_space:
        for el in element:
            flatten_state.append(el)
    #print(flatten_state)    
    maximum = -1*float("inf")
    while delta > epsilon:
        delta = 0
        temp=np.zeros([10,10])
        for curr_state in flatten_state:
            curr_col = int(curr_state/10)
            curr_row = int(curr_state%10)
            v = V[curr_row][curr_col]
            neighborhood = construct_neighborhood(curr_state)
            states_to_visit = neighborhood + [curr_state]
            maximum = -1*float("inf")
            for action in action_set:
                total = 0
                for next_state in states_to_visit:
                    p = transition_probability(curr_state, next_state, action)
                    next_col = int(next_state/10)
                    next_row = int(next_state%10)
                    total += p * (reward[next_row, next_col] + discount_factor * V[next_row][next_col])
                if total>maximum:
                    maximum = total
            temp[curr_row][curr_col] = maximum
            delta = max(delta, np.abs(v - temp[curr_row][curr_col]))
        V=temp
    maximum = -1*float("inf")
    for curr_state in flatten_state:
        curr_col = int(curr_state/10)
        curr_row = int(curr_state%10)
        neighborhood = construct_neighborhood(curr_state)
        states_to_visit = neighborhood + [curr_state]
        maximum = -1*float("inf")
        second_max=-1*float("inf")
        max_index=0
        for action in [0,1,3,2]: # or action set
            total = 0
            for next_state in states_to_visit:
                p = transition_probability(curr_state, next_state, action)
                next_col = int(next_state/10)
                next_row = int(next_state%10)
                total += p * ((reward[next_row, next_col]) + discount_factor * V[next_row][next_col])
            if total>maximum:
                second_max=maximum
                max_index2 = max_index
                
                maximum = total
                max_index = action
                
            elif total>second_max:
                second_maximum = total
                max_index2 = action
        pi[curr_row][curr_col] = max_index
        
    print("once")
    
                
    for curr_state in flatten_state:
        curr_col = int(curr_state/10)
        curr_row = int(curr_state%10)
#         if pi[curr_row][curr_col]==0 and ((curr_row-1)*10+curr_col in flatten_state):
#             if pi[curr_row-1][curr_col]==2:
#                 print("ud",curr_row,curr_col)
#                 pi[curr_row][curr_col]=2
#         if pi[curr_row][curr_col]==1 and ((curr_row)*10+(curr_col-1) in flatten_state):
#             if pi[curr_row][curr_col-1]==3:
#                 print("lr",curr_row,curr_col)
#                 pi[curr_row][curr_col]=3

        if curr_row==0 and pi[curr_row][curr_col]==0:#up
            print(curr_row,curr_col, "changed 0 to 2")
            pi[curr_row][curr_col]=2
        if curr_row==9 and pi[curr_row][curr_col]==2:#down
            print(curr_row,curr_col, "changed 2 to 0")
            pi[curr_row][curr_col]=3
        if curr_col==0 and pi[curr_row][curr_col]==1:#left
            print(curr_row,curr_col, "changed 1 to 3")
            pi[curr_row][curr_col]=2
        if curr_col==9 and pi[curr_row][curr_col]==3:#right
            print(curr_row,curr_col, "changed 3 to 1")
            pi[curr_row][curr_col]=2
    
    for curr_state in flatten_state:
        curr_col = int(curr_state/10)
        curr_row = int(curr_state%10)
        if pi[curr_row][curr_col]==0 and ((curr_row-1)*10+curr_col in flatten_state):
            if pi[curr_row-1][curr_col]==2:
                print("ud",curr_row,curr_col)
                pi[curr_row][curr_col]=2
        if pi[curr_row][curr_col]==1 and ((curr_row)*10+(curr_col-1) in flatten_state):
            if pi[curr_row][curr_col-1]==3:
                print("lr",curr_row,curr_col)
                pi[curr_row][curr_col]=3
    
        
    return pi

In [5]:
num_states=100
num_actions=4
state_space = np.zeros(shape=[10,10])
state_space = [[10.0*i+j for i in range(0,10)] for j in range(0, 10)]
action_set = [0, 1, 2, 3] # top, left, bottom, right
discount_factor = 0.8
w=0.1


In [6]:
prob_matrix = np.zeros(shape=[100,100,4])
#prob_matrix=[[[transition_probability(i,j,k) for i in range(0,100)] for j in range(0, 100)] for k in range(0,4)] 
for k in range(0,4):
    for j in range(0,100):
        for i in range(0,100):
            prob_matrix[i][j][k]=transition_probability(i,j,k);


np.shape(prob_matrix)

(100, 100, 4)

In [7]:
#Q11

from cvxopt import matrix, solvers

def irl(num_states, num_actions, prob_matrix, opt_policy, discount_factor, Rmax, reg):
    
    c = np.zeros(3 * num_states)
    c[num_states:2 * num_states] = -1
    c[2 * num_states:3 * num_states ] = reg
    
    h = np.zeros(2 * num_states * (num_actions - 1) + 4 * num_states)
    h[2 * num_states * (num_actions - 1) + 2 * num_states:2 * num_states * (num_actions - 1) + 4 * num_states ]=Rmax
    
    G = np.zeros([2 * num_states * (num_actions - 1) + 4 * num_states, 3 * num_states])
    
    for i in range(num_states):
        G[2 * num_states * (num_actions - 1) + 2 * num_states+i,i ]=1 #800-900
        G[2 * num_states * (num_actions - 1) + 3 * num_states+i,i ]=-1 #900-1000
        G[2 * num_states * (num_actions - 1) + i , i ]=1 #600-700, 1st col
        G[2 * num_states * (num_actions - 1) + i , 2 * num_states+i ]=-1 #600-700, 3rd col
        G[2 * num_states * (num_actions - 1) + num_states+i,i ]=-1 #700-800, 1st col
        G[2 * num_states * (num_actions - 1) +num_states+ i,2 * num_states+i ]=-1 #700-800, 3rd col
    
#     for i in range(num_states):            ##2nd column for first 300 rows
#         G[i, num_states + i] = 1
#         G[num_states+i,num_states + i]=1
#         G[2*num_states+i,num_states + i]=1
    
    for i in range(num_states):         ##first column for first 600 rows
        a_i = int(opt_policy[i%10][int(i/10)])
        second_term = np.linalg.inv(np.identity(num_states) - discount_factor * prob_matrix[:,:,a_i])

        count = 0
        for a in range(num_actions):
            if (a != a_i):
                G[i * (num_actions - 1) + count, :num_states] = - np.dot(np.asarray(prob_matrix[i,:,a_i]) - np.asarray(prob_matrix[i,:,a]), second_term)
            
                G[num_states * (num_actions - 1) + i * (num_actions - 1) + count, :num_states] = - \
                np.dot(np.asarray(prob_matrix[i,:,a_i]) - np.asarray(prob_matrix[i,:,a]), second_term)
                
                G[i * (num_actions - 1) + count, num_states + i] = 1
                #G[num_states * (num_actions - 1) + i * (num_actions - 1) + count,num_states + i] = 1
                count += 1
            
    sol = solvers.lp(matrix(c), matrix(G), matrix(h))
    return sol

In [8]:
import matplotlib.pyplot as plt
x_plot=[]
reward_1 = np.zeros(shape=[10,10])
reward_1[9][9] = 1
actual_policy=value_iteration_action(reward_1, state_space, action_set, discount_factor)
opt_policy=actual_policy
Rmax=1

In [9]:
#Q11,Q12
max_acc=0
max_reg=0
max_reward=[]
for i in range(0,500):
    x=irl(num_states, num_actions, prob_matrix, opt_policy, discount_factor, Rmax, i/100.0)
    reward_computed=x['x'][:num_states]
    reward_computed=np.reshape(reward_computed,(10,10))
    reward_computed=np.transpose(reward_computed)
    policy_computed = value_iteration_action(reward_computed, state_space, action_set, discount_factor)
    count =0;
    for j in range(10):
        for k in range(10):
            if(policy_computed[j][k]==actual_policy[j][k]):
                count+=1;
    acc=count/100.0;
    if(acc>max_acc):
        max_acc=acc
        max_reg=i/100.0
        max_reward=reward_computed
    x_plot.append(acc);



     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  2e+01  1e+00
 1: -1.1474e+01 -9.9822e+01  4e+02  7e-01  5e+00  1e+00
 2: -2.1292e+01 -7.6033e+01  2e+02  4e-01  3e+00  7e-01
 3: -3.1824e+01 -5.9583e+01  1e+02  2e-01  2e+00  4e-01
 4: -3.8993e+01 -4.8047e+01  4e+01  7e-02  5e-01  9e-02
 5: -4.0771e+01 -4.6492e+01  3e+01  5e-02  3e-01  5e-02
 6: -4.2387e+01 -4.4840e+01  1e+01  2e-02  1e-01  2e-02
 7: -4.3650e+01 -4.4291e+01  3e+00  5e-03  4e-02  4e-03
 8: -4.4038e+01 -4.4311e+01  1e+00  2e-03  2e-02  2e-03
 9: -4.4058e+01 -4.4257e+01  9e-01  2e-03  1e-02  1e-03
10: -4.4154e+01 -4.4234e+01  4e-01  6e-04  5e-03  4e-04
11: -4.4190e+01 -4.4219e+01  1e-01  2e-04  2e-03  1e-04
12: -4.4195e+01 -4.4216e+01  1e-01  2e-04  1e-03  1e-04
13: -4.4193e+01 -4.4215e+01  1e-01  2e-04  1e-03  1e-04
14: -4.4193e+01 -4.4213e+01  1e-01  2e-04  1e-03  1e-04
15: -4.4193e+01 -4.4206e+01  6e-02  1e-04  7e-04  6e-05
16: -4.4197e+01 -4.4201e+01  3e-02  4e-05  3e-04  

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  2e+01  1e+00
 1: -8.8421e+00 -9.2691e+01  3e+02  7e-01  5e+00  1e+00
 2: -1.7375e+01 -6.9715e+01  2e+02  4e-01  3e+00  7e-01
 3: -2.6768e+01 -5.4206e+01  1e+02  2e-01  2e+00  3e-01
 4: -3.2873e+01 -4.3946e+01  5e+01  9e-02  6e-01  1e-01
 5: -3.5606e+01 -4.2093e+01  3e+01  5e-02  4e-01  6e-02
 6: -3.7673e+01 -4.0118e+01  1e+01  2e-02  1e-01  2e-02
 7: -3.8433e+01 -3.9640e+01  5e+00  1e-02  7e-02  8e-03
 8: -3.8709e+01 -3.9367e+01  3e+00  5e-03  4e-02  4e-03
 9: -3.9066e+01 -3.9341e+01  1e+00  2e-03  2e-02  1e-03
10: -3.9200e+01 -3.9308e+01  4e-01  9e-04  6e-03  4e-04
11: -3.9236e+01 -3.9298e+01  2e-01  5e-04  3e-03  3e-04
12: -3.9259e+01 -3.9289e+01  1e-01  2e-04  2e-03  1e-04
13: -3.9255e+01 -3.9287e+01  1e-01  3e-04  2e-03  1e-04
14: -3.9255e+01 -3.9285e+01  1e-01  2e-04  2e-03  1e-04
15: -3.9260e+01 -3.9277e+01  7e-02  1e-04  1e-03  7e-05
16: -3.9266e+01 -3.9270e+01  2e-02  3e-05  2e-04  

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  2e+01  1e+00
 1: -6.3522e+00 -8.5351e+01  3e+02  6e-01  4e+00  1e+00
 2: -1.3711e+01 -6.3291e+01  2e+02  4e-01  3e+00  7e-01
 3: -2.2119e+01 -4.8623e+01  1e+02  2e-01  2e+00  3e-01
 4: -2.7248e+01 -3.9606e+01  5e+01  1e-01  7e-01  1e-01
 5: -3.0905e+01 -3.6655e+01  2e+01  5e-02  3e-01  5e-02
 6: -3.2709e+01 -3.5001e+01  9e+00  2e-02  1e-01  2e-02
 7: -3.3546e+01 -3.4296e+01  3e+00  6e-03  4e-02  5e-03
 8: -3.3809e+01 -3.4122e+01  1e+00  2e-03  2e-02  2e-03
 9: -3.3916e+01 -3.4069e+01  6e-01  1e-03  9e-03  8e-04
10: -3.3955e+01 -3.4043e+01  3e-01  7e-04  5e-03  4e-04
11: -3.3944e+01 -3.4036e+01  4e-01  7e-04  5e-03  5e-04
12: -3.3956e+01 -3.4016e+01  3e-01  5e-04  3e-03  3e-04
13: -3.3973e+01 -3.3994e+01  9e-02  2e-04  1e-03  1e-04
14: -3.3973e+01 -3.3994e+01  9e-02  2e-04  1e-03  9e-05
15: -3.3980e+01 -3.3986e+01  3e-02  5e-05  4e-04  3e-05
16: -3.3981e+01 -3.3983e+01  6e-03  1e-05  8e-05  

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  2e+01  1e+00
 1: -4.5013e+00 -7.9555e+01  3e+02  6e-01  4e+00  1e+00
 2: -1.0823e+01 -5.7799e+01  2e+02  4e-01  3e+00  7e-01
 3: -1.8183e+01 -4.3300e+01  1e+02  2e-01  1e+00  3e-01
 4: -2.2473e+01 -3.5101e+01  5e+01  1e-01  7e-01  1e-01
 5: -2.5740e+01 -3.1879e+01  2e+01  5e-02  3e-01  6e-02
 6: -2.7522e+01 -3.0188e+01  1e+01  2e-02  1e-01  2e-02
 7: -2.8222e+01 -2.9562e+01  5e+00  1e-02  7e-02  9e-03
 8: -2.8679e+01 -2.9245e+01  2e+00  4e-03  3e-02  3e-03
 9: -2.8923e+01 -2.9090e+01  6e-01  1e-03  9e-03  7e-04
10: -2.8942e+01 -2.9071e+01  5e-01  1e-03  7e-03  6e-04
11: -2.8942e+01 -2.9062e+01  5e-01  1e-03  7e-03  5e-04
12: -2.8937e+01 -2.9060e+01  5e-01  1e-03  7e-03  5e-04
13: -2.8949e+01 -2.9034e+01  4e-01  7e-04  5e-03  4e-04
14: -2.8976e+01 -2.9006e+01  1e-01  2e-04  2e-03  1e-04
15: -2.8976e+01 -2.9005e+01  1e-01  2e-04  2e-03  1e-04
16: -2.8988e+01 -2.8993e+01  2e-02  4e-05  3e-04  

19: -2.4875e+01 -2.4875e+01  6e-06  1e-08  8e-08  6e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  2e+01  1e+00
 1: -3.0687e+00 -7.4484e+01  3e+02  6e-01  4e+00  1e+00
 2: -8.4574e+00 -5.2794e+01  2e+02  4e-01  2e+00  7e-01
 3: -1.4329e+01 -3.8773e+01  1e+02  2e-01  1e+00  3e-01
 4: -1.8605e+01 -3.0414e+01  5e+01  9e-02  7e-01  1e-01
 5: -2.1303e+01 -2.6903e+01  2e+01  4e-02  3e-01  5e-02
 6: -2.2750e+01 -2.5348e+01  1e+01  2e-02  1e-01  2e-02
 7: -2.3692e+01 -2.4575e+01  3e+00  7e-03  5e-02  5e-03
 8: -2.3914e+01 -2.4409e+01  2e+00  4e-03  3e-02  3e-03
 9: -2.4088e+01 -2.4336e+01  1e+00  2e-03  1e-02  1e-03
10: -2.4155e+01 -2.4296e+01  6e-01  1e-03  8e-03  6e-04
11: -2.4148e+01 -2.4282e+01  5e-01  1e-03  7e-03  6e-04
12: -2.4172e+01 -2.4246e+01  3e-01  6e-04  4e-03  3e-04
13: -2.4210e+01 -2.4222e+01  5e-02  1e-04  7e-04  5e-05
14: -2.4215e+01 -2.4218e+01  1e-02  2e-05  2e-04  1e-05
15: -2.4217e+01 -2.4218e+0

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  2e+01  1e+00
 1: -1.9343e+00 -6.9776e+01  3e+02  5e-01  4e+00  1e+00
 2: -6.4709e+00 -4.8040e+01  2e+02  3e-01  2e+00  6e-01
 3: -1.1042e+01 -3.3793e+01  9e+01  2e-01  1e+00  3e-01
 4: -1.5282e+01 -2.5195e+01  4e+01  8e-02  5e-01  1e-01
 5: -1.7361e+01 -2.1672e+01  2e+01  3e-02  2e-01  3e-02
 6: -1.8449e+01 -2.0725e+01  9e+00  2e-02  1e-01  1e-02
 7: -1.9240e+01 -2.0051e+01  3e+00  6e-03  4e-02  3e-03
 8: -1.9576e+01 -1.9908e+01  1e+00  3e-03  2e-02  1e-03
 9: -1.9686e+01 -1.9853e+01  6e-01  1e-03  9e-03  7e-04
10: -1.9676e+01 -1.9843e+01  6e-01  1e-03  9e-03  7e-04
11: -1.9702e+01 -1.9814e+01  4e-01  9e-04  6e-03  5e-04
12: -1.9752e+01 -1.9775e+01  9e-02  2e-04  1e-03  9e-05
13: -1.9764e+01 -1.9767e+01  1e-02  3e-05  2e-04  1e-05
14: -1.9765e+01 -1.9766e+01  3e-03  6e-06  4e-05  3e-06
15: -1.9766e+01 -1.9766e+01  9e-04  2e-06  1e-05  9e-07
16: -1.9766e+01 -1.9766e+01  2e-04  4e-07  3e-06  

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  1e+01  1e+00
 1: -9.3212e-01 -6.4748e+01  2e+02  5e-01  3e+00  1e+00
 2: -4.5649e+00 -4.2793e+01  1e+02  3e-01  2e+00  6e-01
 3: -7.5946e+00 -2.8532e+01  8e+01  2e-01  1e+00  3e-01
 4: -1.1797e+01 -1.9105e+01  3e+01  6e-02  4e-01  8e-02
 5: -1.2785e+01 -1.6706e+01  1e+01  3e-02  2e-01  2e-02
 6: -1.3922e+01 -1.5746e+01  7e+00  1e-02  1e-01  1e-02
 7: -1.4464e+01 -1.5317e+01  3e+00  7e-03  4e-02  5e-03
 8: -1.4664e+01 -1.5193e+01  2e+00  4e-03  3e-02  3e-03
 9: -1.4837e+01 -1.5037e+01  7e-01  2e-03  1e-02  8e-04
10: -1.4842e+01 -1.5019e+01  7e-01  1e-03  9e-03  7e-04
11: -1.4902e+01 -1.4973e+01  3e-01  6e-04  4e-03  3e-04
12: -1.4936e+01 -1.4948e+01  5e-02  1e-04  6e-04  5e-05
13: -1.4942e+01 -1.4944e+01  6e-03  1e-05  8e-05  6e-06
14: -1.4943e+01 -1.4943e+01  2e-03  3e-06  2e-05  2e-06
15: -1.4943e+01 -1.4943e+01  3e-04  5e-07  4e-06  3e-07
16: -1.4943e+01 -1.4943e+01  5e-05  1e-07  6e-07  

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -2.8180e+02  2e+03  2e+00  1e+01  1e+00
 1: -1.7658e-01 -6.0014e+01  2e+02  5e-01  3e+00  1e+00
 2: -2.9691e+00 -3.7639e+01  1e+02  3e-01  2e+00  6e-01
 3: -4.6466e+00 -2.3712e+01  7e+01  2e-01  1e+00  2e-01
 4: -8.0099e+00 -1.5506e+01  3e+01  6e-02  4e-01  9e-02
 5: -9.0302e+00 -1.3783e+01  2e+01  4e-02  2e-01  5e-02
 6: -1.0034e+01 -1.1441e+01  5e+00  1e-02  7e-02  9e-03
 7: -1.0418e+01 -1.0885e+01  2e+00  4e-03  2e-02  2e-03
 8: -1.0556e+01 -1.0751e+01  7e-01  2e-03  1e-02  8e-04
 9: -1.0620e+01 -1.0676e+01  2e-01  4e-04  3e-03  2e-04
10: -1.0637e+01 -1.0663e+01  9e-02  2e-04  1e-03  1e-04
11: -1.0644e+01 -1.0656e+01  4e-02  1e-04  6e-04  4e-05
12: -1.0650e+01 -1.0652e+01  7e-03  2e-05  1e-04  7e-06
13: -1.0650e+01 -1.0651e+01  2e-03  4e-06  3e-05  2e-06
14: -1.0651e+01 -1.0651e+01  4e-04  8e-07  5e-06  4e-07
15: -1.0651e+01 -1.0651e+01  5e-05  1e-07  8e-07  5e-08
16: -1.0651e+01 -1.0651e+01  3e-06  6e-09  4e-08  

KeyboardInterrupt: 

In [ ]:
#Q12
lambd = []
for i in range(500):
    lambd.append(i/100.0)
print("max accuracy is:")
print(max_acc)
print("best lambda is:")
print(max_reg)
plt.plot(lambd,x_plot);
plt.show()

In [ ]:
Rmax=1
#Q13


# x=irl(num_states, num_actions, prob_matrix, opt_policy, discount_factor, Rmax, 0.09)
# reward_computed=x['x'][:num_states]
# reward_computed=np.reshape(reward_computed,(10,10))
# reward_computed=np.transpose(reward_computed)
# print(sns.heatmap(reward_1))
# print(sns.heatmap(reward_computed))
plt.clf()
ax = sns.heatmap(reward_1, annot=True)
plt.show()
plt.clf()

# rounded=[]
# for i in max_reward:
#     rounded.append(round(i,2))
plt.figure(figsize=(12,10))
ax1 = sns.heatmap(max_reward, annot=True)
plt.show()

policy_computed = value_iteration_action(max_reward, state_space, action_set, discount_factor)
count =0;
for i in range(10):
    for j in range(10):
        if(policy_computed[i][j]==actual_policy[i][j]):
            count+=1;
acc=count/100.0;
print (acc)


In [ ]:
V = value_iteration_value(max_reward, state_space, action_set, discount_factor)
print(V)

In [ ]:
ax = sns.heatmap(V, annot=True)

In [ ]:
optimal_policy = value_iteration_action(max_reward, state_space, action_set, discount_factor)
print(optimal_policy)

In [ ]:
import sys
arrows = [u'\u2191', u'\u2190',u'\u2193',u'\u2192']
arrow_matrix = np.array(optimal_policy, dtype=object)
# print arrows
for i in range(0, 10):
    for j in range(0, 10):
        index = int(optimal_policy[i][j])
        arrow_matrix[i][j] = arrows[index]
ax = sns.heatmap(optimal_policy, annot=arrow_matrix, fmt='',cbar=False)

In [ ]:
x_plot=[]
reward_2 = np.zeros(shape=[10,10])
reward_2[1, 4:7] = -100
reward_2[1:7, 4] = -100
reward_2[1:4, 6] = -100
reward_2[3, 6:9] = -100
reward_2[3:8, 8] = -100
reward_2[7, 6:9] = -100
reward_2[8, 6] = -100
reward_2[9,9] = 10
actual_policy=value_iteration_action(reward_2, state_space, action_set, discount_factor)
opt_policy=actual_policy
Rmax=10

In [ ]:
#Q18,Q19
max_acc=0
max_reg=0
max_reward=[]
for i in range(0,500):
    x=irl(num_states, num_actions, prob_matrix, opt_policy, discount_factor, Rmax, i/100.0)
    reward_computed=x['x'][:num_states]
    reward_computed=np.reshape(reward_computed,(10,10))
    reward_computed=np.transpose(reward_computed)
    policy_computed = value_iteration_action(reward_computed, state_space, action_set, discount_factor)
    count =0;
    for j in range(10):
        for k in range(10):
            if(policy_computed[j][k]==actual_policy[j][k]):
                count+=1;
    acc=count/100.0;
    if(acc>max_acc):
        max_acc=acc
        max_reg=i/100.0
        max_reward=reward_computed
    x_plot.append(acc);

In [ ]:
#Q19
lambd = []
for i in range(500):
    lambd.append(i/100.0)
print("max accuracy is:")
print(max_acc)
print("best lambda is:")
print(max_reg)
plt.plot(lambd,x_plot);
plt.show()

In [ ]:
#Q20
plt.clf()
plt.figure(figsize=(12,10))
ax = sns.heatmap(reward_2, annot=True)
plt.show()
plt.clf()
plt.figure(figsize=(12,10))
ax1 = sns.heatmap(max_reward, annot=True)
plt.show()

policy_computed = value_iteration_action(max_reward, state_space, action_set, discount_factor)
print policy_computed

count =0;
for i in range(10):
    for j in range(10):
        if(policy_computed[i][j]==actual_policy[i][j]):
            count+=1;
acc=count/100.0;
print acc



In [ ]:
plt.clf()
plt.figure(figsize=(12,10))
ax1 = sns.heatmap(max_reward, annot=True)
plt.show()
print (max_reward)